In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
from functools import reduce
from datetime import datetime, time
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import math
import folium

In [2]:
spark = SparkSession.builder.master('local[*]') \
    .appName('Métricas') \
    .getOrCreate()

23/09/30 12:27:06 WARN Utils: Your hostname, DESKTOP-N81ME1R resolves to a loopback address: 127.0.1.1; using 172.24.108.240 instead (on interface eth0)
23/09/30 12:27:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/30 12:27:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/30 12:27:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
type(spark)

pyspark.sql.session.SparkSession

<p>1 – Quantidade de ônibus em operação</p>

In [4]:
directory_path = "/home/rodrigo/projetos/monitoramento_sptrans/data/datalake/prata/operacao_agrupada/"
file_list = os.listdir(directory_path)
dataframes_agrupados = []
for item in file_list:
    if item.endswith('.parquet'):
        caminho_completo = os.path.join(directory_path, item)
        df = spark.read.parquet(caminho_completo)
        dataframes_agrupados.append(df)

In [5]:
def union_all(dfs):
    return reduce(DataFrame.unionAll, dfs)

In [6]:
dataframes_agrupados_completo = union_all(dataframes_agrupados)
dataframes_agrupados_completo.show(truncate=False)

+-------+-----------+-------------------------------------------+-------------------------------------------+--------+-------------+-----------------+--------------------+-----------------+---------------------+----------------------+-----------------+
|LINHA  |CODIGO_AREA|CONSOCIO                                   |EMPRESA                                    |HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM  |LETREIRO_DESTINO     |QTDE_VEICULOS_OPERACAO|DATA_EXTRACAO_API|
+-------+-----------+-------------------------------------------+-------------------------------------------+--------+-------------+-----------------+--------------------+-----------------+---------------------+----------------------+-----------------+
|3054-10|4          |PÊSSEGO TRANSPORTES LTDA                   |PÊSSEGO TRANSPORTES LTDA                   |08:50   |2023-09-15   |3054-10          |33743               |HOSP. SAPOPEMBA  |JD. PALANQUE         |6                     |2023-09

In [7]:
dataframes_agrupados_completo.count()

1083996

In [8]:
lista_tempo = dataframes_agrupados_completo.select('DATA_EXTRACAO')\
        .distinct().rdd.flatMap(lambda linha: [linha.DATA_EXTRACAO]).collect()

In [9]:
lista_tempo

[datetime.date(2023, 9, 15),
 datetime.date(2023, 9, 16),
 datetime.date(2023, 9, 17),
 datetime.date(2023, 9, 18)]

In [10]:
max(lista_tempo)

datetime.date(2023, 9, 18)

In [11]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)
 |-- DATA_EXTRACAO_API: date (nullable = true)



In [12]:
dataframes_agrupados_completo = dataframes_agrupados_completo.drop('DATA_EXTRACAO_API')
dataframes_agrupados_completo.show()

+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+-----------------+--------------------+----------------------+
|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|  LETREIRO_ORIGEM|    LETREIRO_DESTINO|QTDE_VEICULOS_OPERACAO|
+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+-----------------+--------------------+----------------------+
|3054-10|          4|PÊSSEGO TRANSPORT...|PÊSSEGO TRANSPORT...|   08:50|   2023-09-15|          3054-10|               33743|  HOSP. SAPOPEMBA|        JD. PALANQUE|                     6|
|6026-10|          6|TRANSWOLFF TRANSP...|TRANSWOLFF TRANSP...|   08:50|   2023-09-15|          6026-10|                  87| TERM. STO. AMARO|          JD. ICARAÍ|                     5|
|857A-10|          7|      CONSÓRCIO KBPX|KBPX ADMINISTRAÇÃ.

In [13]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)



In [14]:
dataframes_agrupados_completo.columns

['LINHA',
 'CODIGO_AREA',
 'CONSOCIO',
 'EMPRESA',
 'HORA_API',
 'DATA_EXTRACAO',
 'LETREIRO_COMPLETO',
 'CODIGO_IDENTIFICADOR',
 'LETREIRO_ORIGEM',
 'LETREIRO_DESTINO',
 'QTDE_VEICULOS_OPERACAO']

In [15]:
@f.udf(returnType=t.StringType())
def turno(hora: str):
    hora_formatada = datetime.strptime(hora, '%H:%M').time()
    if 0 <= hora_formatada.hour < 6:
        return 'Madrugada'
    elif 6 <= hora_formatada.hour < 12:
        return 'Manhã'
    elif 12 <= hora_formatada.hour < 18:
        return 'Tarde'
    else:
        return 'Noite'
        

In [16]:
turno('23:55')

Column<'turno(23:55)'>

In [17]:
dataframes_agrupados_completo = dataframes_agrupados_completo.withColumn('TURNO', turno(f.col('HORA_API'))) 

df_filter = dataframes_agrupados_completo.select(
    dataframes_agrupados_completo.CODIGO_AREA,
    dataframes_agrupados_completo.CONSOCIO,
    dataframes_agrupados_completo.EMPRESA,
    dataframes_agrupados_completo.TURNO,
    dataframes_agrupados_completo.DATA_EXTRACAO ,
    dataframes_agrupados_completo.LETREIRO_ORIGEM,
    dataframes_agrupados_completo.LETREIRO_DESTINO,
    dataframes_agrupados_completo.HORA_API,
    dataframes_agrupados_completo.LETREIRO_COMPLETO,
    dataframes_agrupados_completo.QTDE_VEICULOS_OPERACAO,
    dataframes_agrupados_completo.CODIGO_IDENTIFICADOR,
    ) \
    .filter(
        (dataframes_agrupados_completo.DATA_EXTRACAO == '2023-09-16' )  &
        
        (dataframes_agrupados_completo.TURNO == 'Noite') &
        (dataframes_agrupados_completo.LETREIRO_COMPLETO == '9047-10')
        ) \
        
df_filter.show(truncate=False)

+-----------+-----------------------+---------------------------+-----+-------------+---------------+----------------+--------+-----------------+----------------------+--------------------+
|CODIGO_AREA|CONSOCIO               |EMPRESA                    |TURNO|DATA_EXTRACAO|LETREIRO_ORIGEM|LETREIRO_DESTINO|HORA_API|LETREIRO_COMPLETO|QTDE_VEICULOS_OPERACAO|CODIGO_IDENTIFICADOR|
+-----------+-----------------------+---------------------------+-----+-------------+---------------+----------------+--------+-----------------+----------------------+--------------------+
|1          |CONSÓRCIO TRANSNOROESTE|NORTE BUSS TRANSPORTES S.A.|Noite|2023-09-16   |LAPA           |JD. PAULISTANO  |18:05   |9047-10          |4                     |33651               |
|1          |CONSÓRCIO TRANSNOROESTE|NORTE BUSS TRANSPORTES S.A.|Noite|2023-09-16   |LAPA           |JD. PAULISTANO  |18:05   |9047-10          |3                     |883                 |
|1          |CONSÓRCIO TRANSNOROESTE|NORTE BUSS TR

In [18]:
df_filter.groupBy(['CODIGO_AREA']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULOS_OPERACAO')
    ) \
    .orderBy('CODIGO_AREA') \
    .show(truncate=False)
df_filter.toPandas()

+-----------+----------------------------+
|CODIGO_AREA|QUANTIDADE_VEICULOS_OPERACAO|
+-----------+----------------------------+
|1          |357                         |
+-----------+----------------------------+



,CODIGO_AREA,CONSOCIO,EMPRESA,TURNO,DATA_EXTRACAO,LETREIRO_ORIGEM,LETREIRO_DESTINO,HORA_API,LETREIRO_COMPLETO,QTDE_VEICULOS_OPERACAO,CODIGO_IDENTIFICADOR
0,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:05,9047-10,4,33651
1,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:05,9047-10,3,883
2,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:10,9047-10,5,33651
3,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:10,9047-10,3,883
4,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,18:15,9047-10,3,33651
...,...,...,...,...,...,...,...,...,...,...,...
132,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,23:45,9047-10,2,33651
133,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,23:45,9047-10,1,883
134,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,23:50,9047-10,2,33651
135,1,CONSÓRCIO TRANSNOROESTE,NORTE BUSS TRANSPORTES S.A.,Noite,2023-09-16,LAPA,JD. PAULISTANO,23:55,9047-10,1,33651


In [19]:
df_filter.groupBy(['EMPRESA']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULO_OPERACAO')
    ) \
    .show(truncate=False)

+---------------------------+---------------------------+
|EMPRESA                    |QUANTIDADE_VEICULO_OPERACAO|
+---------------------------+---------------------------+
|NORTE BUSS TRANSPORTES S.A.|357                        |
+---------------------------+---------------------------+



In [20]:
df_filter.groupBy(['DATA_EXTRACAO', 'EMPRESA']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULO_OPERACAO') 
    ) \
    .show(truncate=False)

+-------------+---------------------------+---------------------------+
|DATA_EXTRACAO|EMPRESA                    |QUANTIDADE_VEICULO_OPERACAO|
+-------------+---------------------------+---------------------------+
|2023-09-16   |NORTE BUSS TRANSPORTES S.A.|357                        |
+-------------+---------------------------+---------------------------+



In [21]:
df_filter.groupBy(['DATA_EXTRACAO']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULO_OPERACAO') 
    ) \
    .show(truncate=False)

+-------------+---------------------------+
|DATA_EXTRACAO|QUANTIDADE_VEICULO_OPERACAO|
+-------------+---------------------------+
|2023-09-16   |357                        |
+-------------+---------------------------+



In [22]:
directory_path = "/home/rodrigo/projetos/monitoramento_sptrans/data/datalake/prata/operacao_desagrupada"
file_list = os.listdir(directory_path)
dataframes_desagrupados = []
for item in file_list:
    if item.endswith('.parquet'):
        caminho_completo = os.path.join(directory_path, item)

        df = spark.read.parquet(caminho_completo)
        dataframes_desagrupados.append(df)



In [23]:
dataframes_desagrupados_completo = union_all(dataframes_desagrupados)
dataframes_desagrupados_completo = dataframes_desagrupados_completo.drop('LINHA')
dataframes_desagrupados_completo = dataframes_desagrupados_completo.withColumn('TURNO', turno(f.col('HORA_API')))
dataframes_desagrupados_completo.show(truncate=False)

+-----------+-------------------------------------------+-------------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|CODIGO_AREA|CONSOCIO                                   |EMPRESA                                    |DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|LATITUDE           |LONGITUDE          |DATA_EXTRACAO_API|TURNO|
+-----------+-------------------------------------------+-------------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|4          |PÊSSEGO TRANSPORTES LTDA                   |PÊSSEGO TRANSPORTES LTDA                   |2023-09-15 

In [24]:
dataframe_prefixo_onibus = dataframes_desagrupados_completo \
    .select(
        dataframes_desagrupados_completo.HORA_API,
        dataframes_desagrupados_completo.PREFIXO_ONIBUS,
        dataframes_desagrupados_completo.CODIGO_IDENTIFICADOR_LINHA,
        dataframes_desagrupados_completo.LETREIRO_COMPLETO,
        dataframes_desagrupados_completo.LATITUDE,
        dataframes_desagrupados_completo.LONGITUDE,
    ) \
    .filter(
        (dataframes_desagrupados_completo.PREFIXO_ONIBUS == '55393') &
        (dataframes_desagrupados_completo.TURNO == 'Noite') &
        (dataframes_desagrupados_completo.DATA_EXTRACAO == '2023-09-18')
    )



dataframe_prefixo_onibus.show(33)

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|   18:00|         55393|                     34448|          4726-10|        -23.5661365|        -46.5800915|
|   18:05|         55393|                     34448|          4726-10|-23.562075749999998|         -46.592059|
|   18:10|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|
|   18:15|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|
|   18:20|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|
|   18:25|         55393|                     34448|          4726-10|-23.563520500000003|        -46.5915385|
|

In [25]:
def haversine(lat1, lon1, lat2, lon2):

    r = 6371.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

 
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = r * c

    return distance

@pandas_udf(DoubleType())
def haversine_udf(lat, lon):
    distances = []
    for i in range(len(lat)):
        
        if i == 0:
            distances.append(0.0)  
        else:
            distance = haversine(lat[i], lon[i], lat[i - 1], lon[i - 1])
            distances.append(distance)
    return pd.Series(distances)

df= dataframe_prefixo_onibus.withColumn("DISTANCIA", haversine_udf(dataframe_prefixo_onibus["LATITUDE"], dataframe_prefixo_onibus["LONGITUDE"]))


df.show(33)


soma_distancias = df.selectExpr("sum(DISTANCIA)").collect()[0][0]
print(f'Soma das distâncias: {soma_distancias:.2f} km')



+--------+--------------+--------------------------+-----------------+-------------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|          DISTANCIA|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+-------------------+
|   18:00|         55393|                     34448|          4726-10|        -23.5661365|        -46.5800915|                0.0|
|   18:05|         55393|                     34448|          4726-10|-23.562075749999998|         -46.592059| 1.3006534951896462|
|   18:10|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|0.14496006505067272|
|   18:15|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|                0.0|
|   18:20|         55393|                     34448|          4726-10|        -23.5

Soma das distâncias: 14.60 km


In [26]:
dataframe_prefixo_onibus.select(
    dataframe_prefixo_onibus.HORA_API
).distinct().show()

+--------+
|HORA_API|
+--------+
|   18:05|
|   18:32|
|   19:15|
|   19:10|
|   19:35|
|   20:00|
|   18:40|
|   18:15|
|   19:05|
|   18:35|
|   19:20|
|   20:15|
|   20:20|
|   18:10|
|   18:45|
|   19:00|
|   19:55|
|   19:50|
|   18:00|
|   19:30|
+--------+
only showing top 20 rows



In [138]:
df_selecao_rota_linha = spark.read \
    .options(delimiter=',', header=True, inferSchema='True') \
        .csv('../data/datalake/bronze/arquivos_gtfs/routes.txt', )
opcoes = df_selecao_rota_linha.rdd.flatMap(lambda linha :[[( linha.route_short_name, linha.route_long_name), (linha.route_color)]]).collect()
opcoes

[[('1012-10', 'Term. Jd. Britania - Jd. Monte Belo'), '509E2F'],
 [('1012-21', 'Term. Jd. Britânia - Jd. Rosinha'), '509E2F'],
 [('1015-10', 'Term. Jd. Britânia - Chác. Maria Trindade'), '509E2F'],
 [('1016-10', 'Cem. Do Horto - Shop. Center Norte'), '002F6C'],
 [('1017-10', 'Perus - Conexão Vl. Iório'), '509E2F'],
 [('1018-10', 'Vl. Rosa - Metrô Santana'), '002F6C'],
 [('1019-10', 'Sol Nascente - Term. Pirituba'), '509E2F'],
 [('1020-10', 'Perus - Conexão Vl. Iório '), '509E2F'],
 [('1021-10', 'Cohab Brasilândia - Term. Pirituba'), '509E2F'],
 [('1023-10', 'Vl. Pirituba - Cptm Pirituba'), '509E2F'],
 [('1024-10', 'Jd. Carombé - Conexão Petrônio Portela'), '509E2F'],
 [('1025-10', 'Jd. Carombé - Conexão Petrônio Portela'), '509E2F'],
 [('1026-10', 'Vl. Iara - Conexão Petrônio Portela'), '509E2F'],
 [('1034-10', 'Vl. Iara - Conexão Vl. Iório'), '509E2F'],
 [('1036-10', 'Cohab Brasilândia - Conexão Vl. Iório'), '509E2F'],
 [('106A-10', 'Metrô Santana - Itaim Bibi'), '002F6C'],
 [('107T-1

In [127]:
df_selecao_linha = spark.read.options(delimiter=',', header=True, inferSchema='True') \
        .csv('../data/datalake/bronze/arquivos_gtfs/trips.txt', )

df_selecao_linha = df_selecao_linha.select(
    df_selecao_linha.shape_id
).filter(
    df_selecao_linha.route_id == '1177-31' 
)

shape_id = df_selecao_linha.rdd.flatMap(lambda linha: [linha.shape_id]).collect()
shape_id

[70860, 70861]

In [128]:
df_selecao_linha.show()

+--------+
|shape_id|
+--------+
|   70860|
|   70861|
+--------+



In [101]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType

In [129]:
df_trajeto_linhas = spark.read.options(delimiter=',', header=True).csv('../data/datalake/bronze/arquivos_gtfs/shapes.txt', )
df_linha_filter = df_trajeto_linhas.filter(df_trajeto_linhas.shape_id.isin(shape_id) )
df_linha_filter = df_linha_filter.withColumn('shape_pt_lat', col('shape_pt_lat').cast(FloatType()))
df_linha_filter = df_linha_filter.withColumn('shape_pt_lon', col('shape_pt_lon').cast(FloatType()))
posicao = df_linha_filter.rdd.flatMap(lambda linha: [[linha.shape_pt_lat, linha.shape_pt_lon]]).collect()
posicao
# df_linha_filter.show()

[[-23.517658233642578, -46.475955963134766],
 [-23.517597198486328, -46.47596740722656],
 [-23.517499923706055, -46.475982666015625],
 [-23.51727294921875, -46.476016998291016],
 [-23.517213821411133, -46.476043701171875],
 [-23.51719093322754, -46.47605895996094],
 [-23.517139434814453, -46.476097106933594],
 [-23.517099380493164, -46.476139068603516],
 [-23.517038345336914, -46.47627639770508],
 [-23.51704978942871, -46.47631072998047],
 [-23.517059326171875, -46.47633743286133],
 [-23.517074584960938, -46.476356506347656],
 [-23.517099380493164, -46.476375579833984],
 [-23.518524169921875, -46.47611618041992],
 [-23.51894187927246, -46.47603988647461],
 [-23.519512176513672, -46.47593307495117],
 [-23.519733428955078, -46.47589111328125],
 [-23.520050048828125, -46.47582244873047],
 [-23.521953582763672, -46.47549819946289],
 [-23.523469924926758, -46.4752197265625],
 [-23.523645401000977, -46.475181579589844],
 [-23.523706436157227, -46.47517013549805],
 [-23.523801803588867, -46.4

In [83]:
posicao[0]

[-23.431827545166016, -46.78715133666992]

In [130]:
mapa_linhas = folium.Map(location = posicao[0], 
                         zoom_start = 12, 
                         control_scale = True)



for i in range(len(posicao)-1):
    folium.PolyLine(locations = [[posicao[i][0], posicao[i][1]], [posicao[i+1][0], posicao[i+1][1]]],
                   color = 'navy').add_to(mapa_linhas)
    
mapa_linhas.add_child(folium.LatLngPopup())

mapa_linhas